In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-science-salaries-2023/ds_salaries.csv


In [2]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import normalize, scale
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
df = pd.read_csv('/kaggle/input/data-science-salaries-2023/ds_salaries.csv')
#del df['Unnamed: 0']

In [242]:
#yeni
#df = pd.read_csv('/kaggle/input/data-science-salaries-2023/ds_salaries.csv')

In [7]:
g = ["Data Engineer", "Data Scientist","Data Analyst", "Machine Learning Engineer" , "Analytics Engineer", "Data Architect"]
new=[col for col in df.index if df.job_title[col] in g]

df=df.loc[new,]
df = df.reset_index()
del df["index"]
df.shape

(2985, 11)

In [8]:
df.company_size = df.company_size.replace({'S':1 ,'M': 2, 'L': 3})
df['remote_ratio'] = df['remote_ratio'].replace({100: 3, 50: 2, 0: 1})
df['experience_level'] = df['experience_level'].replace({'EN': 1, 'MI': 2, 'SE': 3, 'EX': 4})
df['employment_type'] = df['employment_type'].replace({'FL':1 ,'PT': 2, 'FT': 3, 'CT': 4,})

In [9]:
b=df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           2985 non-null   int64 
 1   experience_level    2985 non-null   int64 
 2   employment_type     2985 non-null   int64 
 3   job_title           2985 non-null   object
 4   salary              2985 non-null   int64 
 5   salary_currency     2985 non-null   object
 6   salary_in_usd       2985 non-null   int64 
 7   employee_residence  2985 non-null   object
 8   remote_ratio        2985 non-null   int64 
 9   company_location    2985 non-null   object
 10  company_size        2985 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 256.6+ KB


In [10]:
x=df
x = x.astype(str)
x.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           2985 non-null   object
 1   experience_level    2985 non-null   object
 2   employment_type     2985 non-null   object
 3   job_title           2985 non-null   object
 4   salary              2985 non-null   object
 5   salary_currency     2985 non-null   object
 6   salary_in_usd       2985 non-null   object
 7   employee_residence  2985 non-null   object
 8   remote_ratio        2985 non-null   object
 9   company_location    2985 non-null   object
 10  company_size        2985 non-null   object
dtypes: object(11)
memory usage: 256.6+ KB


In [11]:
x["cluster"] = ['_'.join(i) for i in x.drop(["salary","salary_in_usd"], axis=1).values]
x = x['cluster']
#x = x.reset_index()

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           2985 non-null   int64 
 1   experience_level    2985 non-null   int64 
 2   employment_type     2985 non-null   int64 
 3   job_title           2985 non-null   object
 4   salary              2985 non-null   int64 
 5   salary_currency     2985 non-null   object
 6   salary_in_usd       2985 non-null   int64 
 7   employee_residence  2985 non-null   object
 8   remote_ratio        2985 non-null   int64 
 9   company_location    2985 non-null   object
 10  company_size        2985 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 256.6+ KB


In [13]:
df["cluster"]=x

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           2985 non-null   int64 
 1   experience_level    2985 non-null   int64 
 2   employment_type     2985 non-null   int64 
 3   job_title           2985 non-null   object
 4   salary              2985 non-null   int64 
 5   salary_currency     2985 non-null   object
 6   salary_in_usd       2985 non-null   int64 
 7   employee_residence  2985 non-null   object
 8   remote_ratio        2985 non-null   int64 
 9   company_location    2985 non-null   object
 10  company_size        2985 non-null   int64 
 11  cluster             2985 non-null   object
dtypes: int64(7), object(5)
memory usage: 280.0+ KB


In [15]:
df_new=df

In [16]:
df_new['country_median_salary'] = df_new.groupby(['work_year','experience_level','employment_type','job_title','employee_residence','remote_ratio','company_location','company_size'])['salary'].transform('median')
df_new['country_mean_salary'] = df_new.groupby(['work_year','experience_level','employment_type','job_title','employee_residence','remote_ratio','company_location','company_size'])['salary'].transform('mean')
df_new['country_min_salary'] = df_new.groupby(['work_year','experience_level','employment_type','job_title','employee_residence','remote_ratio','company_location','company_size'])['salary'].transform('min')  
df_new['country_max_salary'] = df_new.groupby(['work_year','experience_level','employment_type','job_title','employee_residence','remote_ratio','company_location','company_size'])['salary'].transform('max')

In [17]:
df_new.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,cluster,country_median_salary,country_mean_salary,country_min_salary,country_max_salary
0,2023,3,3,Data Scientist,175000,USD,175000,CA,3,CA,2,2023_3_3_Data Scientist_USD_CA_3_CA_2,147500.000,147500.000,120000,175000
1,2023,3,3,Data Scientist,120000,USD,120000,CA,3,CA,2,2023_3_3_Data Scientist_USD_CA_3_CA_2,147500.000,147500.000,120000,175000
2,2023,3,3,Data Scientist,219000,USD,219000,CA,1,CA,2,2023_3_3_Data Scientist_USD_CA_1_CA_2,183000.000,178508.333,110000,258000
3,2023,3,3,Data Scientist,141000,USD,141000,CA,1,CA,2,2023_3_3_Data Scientist_USD_CA_1_CA_2,183000.000,178508.333,110000,258000
4,2023,3,3,Data Scientist,147100,USD,147100,US,1,US,2,2023_3_3_Data Scientist_USD_US_1_US_2,161000.000,169193.949,70000,370000


In [18]:
a = []
for (i, j) in zip(df_new['employee_residence'], df_new['company_location']):
    if i == j:
        a.append('Yes')
    else:
        a.append('No')

df_new['work_own_country'] = a
df_new.head()
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   work_year              2985 non-null   int64  
 1   experience_level       2985 non-null   int64  
 2   employment_type        2985 non-null   int64  
 3   job_title              2985 non-null   object 
 4   salary                 2985 non-null   int64  
 5   salary_currency        2985 non-null   object 
 6   salary_in_usd          2985 non-null   int64  
 7   employee_residence     2985 non-null   object 
 8   remote_ratio           2985 non-null   int64  
 9   company_location       2985 non-null   object 
 10  company_size           2985 non-null   int64  
 11  cluster                2985 non-null   object 
 12  country_median_salary  2985 non-null   float64
 13  country_mean_salary    2985 non-null   float64
 14  country_min_salary     2985 non-null   int64  
 15  coun

In [19]:
cats = df_new.select_dtypes(include="object").columns
def one_hot_encoder(dataframe, categorical_cols, drop_first=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]

df2 = one_hot_encoder(df_new, cats)

df2.head()

,work_year,experience_level,employment_type,salary,salary_in_usd,remote_ratio,company_size,country_median_salary,country_mean_salary,country_min_salary,country_max_salary,job_title_Data Analyst,job_title_Data Architect,job_title_Data Engineer,job_title_Data Scientist,job_title_Machine Learning Engineer,salary_currency_BRL,salary_currency_CAD,salary_currency_CHF,salary_currency_CLP,salary_currency_EUR,salary_currency_GBP,salary_currency_HKD,salary_currency_HUF,salary_currency_INR,salary_currency_JPY,salary_currency_MXN,salary_currency_PLN,salary_currency_SGD,salary_currency_THB,salary_currency_TRY,salary_currency_USD,employee_residence_AM,employee_residence_AR,employee_residence_AT,employee_residence_AU,employee_residence_BE,employee_residence_BG,employee_residence_BO,employee_residence_BR,employee_residence_CA,employee_residence_CF,employee_residence_CH,employee_residence_CL,employee_residence_CN,employee_residence_CO,employee_residence_CR,employee_residence_CY,employee_residence_DE,employee_residence_DO,employee_residence_DZ,employee_residence_ES,employee_residence_FI,employee_residence_FR,employee_residence_GB,employee_residence_GR,employee_residence_HK,employee_residence_HR,employee_residence_HU,employee_residence_ID,employee_residence_IE,employee_residence_IN,employee_residence_IR,employee_residence_IT,employee_residence_JP,employee_residence_KW,employee_residence_LT,employee_residence_LV,employee_residence_MT,employee_residence_MX,employee_residence_NG,employee_residence_NL,employee_residence_PH,employee_residence_PK,employee_residence_PL,employee_residence_PR,employee_residence_PT,employee_residence_RO,employee_residence_RS,employee_residence_RU,employee_residence_SG,employee_residence_SI,employee_residence_TH,employee_residence_TR,employee_residence_UA,employee_residence_US,employee_residence_UZ,employee_residence_VN,company_location_AM,company_location_AR,company_location_AT,company_location_AU,company_location_BE,company_location_BO,company_location_BR,company_location_CA,company_location_CF,company_location_CH,company_location_CL,company_location_CO,company_location_DE,company_location_DZ,company_location_EE,company_location_ES,company_location_FI,company_location_FR,company_location_GB,company_location_GR,company_location_HK,company_location_HR,company_location_HU,company_location_ID,company_location_IE,company_location_IL,company_location_IN,company_location_IR,company_location_IT,company_location_JP,company_location_LT,company_location_LU,company_location_LV,company_location_MT,company_location_MX,company_location_MY,company_location_NG,company_location_NL,company_location_PH,company_location_PK,company_location_PL,company_location_PR,company_location_PT,company_location_RO,company_location_SG,company_location_SI,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,cluster_2020_1_3_Data Analyst_EUR_PT_3_PT_2,cluster_2020_1_3_Data Analyst_INR_IN_1_IN_1,cluster_2020_1_3_Data Analyst_USD_NG_3_NG_1,cluster_2020_1_3_Data Analyst_USD_US_3_US_3,cluster_2020_1_3_Data Engineer_EUR_PK_3_DE_3,cluster_2020_1_3_Data Engineer_INR_IN_3_IN_3,cluster_2020_1_3_Data Engineer_JPY_JP_3_JP_1,cluster_2020_1_3_Data Scientist_EUR_DE_1_DE_1,cluster_2020_1_3_Data Scientist_EUR_DE_2_DE_1,cluster_2020_1_3_Data Scientist_EUR_FR_1_FR_1,cluster_2020_1_3_Data Scientist_EUR_FR_1_FR_2,cluster_2020_1_3_Data Scientist_USD_US_3_US_1,cluster_2020_1_3_Machine Learning Engineer_USD_US_2_US_3,cluster_2020_1_3_Machine Learning Engineer_USD_US_3_US_1,cluster_2020_2_3_Data Analyst_EUR_FR_2_FR_3,cluster_2020_2_3_Data Analyst_USD_PK_2_PK_3,cluster_2020_2_3_Data Analyst_USD_US_3_US_3,cluster_2020_2_3_Data Engineer_EUR_AT_2_AT_3,cluster_2020_2_3_Data Engineer_EUR_DE_3_DE_1,cluster_2020_2_3_Data Engineer_EUR_FR_2_FR_3,cluster_2020_2_3_Data Engineer_GBP_GB_2_GB_3,cluster_2020_2_3_Data Engineer_USD_ES_3_US_2,cluster_2020_2_3_Data Engineer_USD_US_3_US_3,cluster_2020_2_3_Data Scientist_EUR_DE_1_DE_3,cluster_2020_2_3_Data

In [20]:
#del df2["Unnamed: 0"]
del df2["country_mean_salary"]

In [21]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
scaler = MinMaxScaler()
#scaler_s = StandardScaler()
#scaler_r = RobustScaler()

df2= df2.drop(["salary"], axis=1)
y=df2['salary_in_usd']
X=df2.drop(['salary_in_usd'],axis=1)
X = scaler.fit_transform(X)
y= y.values.reshape(-1,1)
y = np.log(y)


def regression_(x, y):
    lr = LinearRegression()
    rf=RandomForestRegressor()
    lg = LGBMRegressor()
    #ct = CatBoostRegressor()
    r = Ridge()
    l = Lasso()
    e = ElasticNet()
    kn = KNeighborsRegressor()
    et = ExtraTreeRegressor()
    gb = GradientBoostingRegressor()
    dt = DecisionTreeRegressor()
    xgb = XGBRegressor()

    algos = [lr,rf,lg ,r, l, e, kn, et, gb, dt,xgb]
    algos_names = ['LinearRegressor',"rf","lgbm", 'Ridge', 'Lasso', 'ElasticNet', 'KNeighbors', 'ExtraTree', 'GradientBoosting',
                   'DecisionTree','XGB']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)

    r_score = []
    mse = []
    mae = []

    result = pd.DataFrame(columns=['R_square', 'MSE', 'MAE'], index=algos_names)

    for algo in algos:
        pred = algo.fit(x_train, y_train).predict(x_test)
        r_score.append(r2_score(y_test, pred))
        mse.append(mean_squared_error(y_test, pred) ** .5)
        mae.append(mean_absolute_error(y_test, pred))

    result.R_square = r_score
    result.MSE = mse
    result.MAE = mae

    return result.sort_values('R_square', ascending=False)


In [258]:
 regression_(X, y)

,R_square,MSE,MAE
GradientBoosting,0.701,0.365,0.224
XGB,0.700,0.367,0.223
rf,0.691,0.372,0.225
lgbm,0.689,0.373,0.262
DecisionTree,0.631,0.406,0.252
ExtraTree,0.619,0.413,0.289
Ridge,0.570,0.439,0.283
KNeighbors,0.387,0.524,0.323
Lasso,-0.000,0.669,0.478
ElasticNet,-0.000,0.669,0.478


# HyperParameter Optimization

# gbm

In [22]:
gbm_model=GradientBoostingRegressor()

In [23]:
gbm_params = { 
    'n_estimators': [100, 500],
    'subsample': [1,0.5],
    'max_depth' : [3,8],
    "learning_rate": [0.01,0.1]
}

In [24]:
gbm_best_grid = GridSearchCV(gbm_model,gbm_params, cv=5,n_jobs=-1, verbose=True).fit(X,y)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [25]:
gbm_best_grid.best_params_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500, 'subsample': 0.5}

In [26]:
final=gbm_model.set_params(**gbm_best_grid.best_params_,random_state=17).fit(X,y)

In [27]:
final_test=final.predict(x_test)

NameError: name 'x_test' is not defined

In [28]:
r2_score(y_test,final_test)

NameError: name 'y_test' is not defined

In [265]:
gbm_best_grid.best_params_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500, 'subsample': 0.5}